# Homework 3: Mining Data Streams

### Description
You are to study and implement a streaming graph processing algorithm described in one of the above papers of your choice. To accomplish your task, you are to perform the following two steps.

First, implement the reservoir sampling or the Flajolet-Martin algorithm used in the graph algorithm presented in the paper you have selected;
Second, implement the streaming graph algorithm presented in the paper that uses the algorithm implemented in the first step. 
To ensure that your implementation is correct, you are to test your implementation with some of the publicly available graph datasets (find a link below) and present your test results in a report.

Implementation can be done using any data processing framework that includes support for stream (streaming graph) processing, such as Apache Spark, Apache Flink, or no framework, e.g., in Java, Python, or another language of your choice. 

### Description

***1.*** Reservoir Sampling is an algorithm with the task to approximate the distribution of elements in a stream. Specifically of a stream where the number of elements is unknown. This is done by randomly add abitrary elements from the stream which makes the algorithm efficient. 

***2.*** In this program, I'll try to use Trièst-Base  to be able to approximate how many triangles there are in a graph structure. This approximation is done by feeding in edges of the graph into the algorithm where at a random time t an element is added to the sample size (representing the true distribution). Furthermore, when an element is added to the samples one element is deleted to sustain the sample size of M. 

## Code implementation

### Libraries

In [1]:
import operator
import random
import time

### TRIÈST - BASE


In [19]:
class TriestBase:
  def __init__(self, M):

    self.M = M # postitive paramiter M (M numbers of edges)
    self.t = 0 # time instant t
    self.global_count = 0 # estimation of globle edges
    self.local_count = dict() # local counters
    self.Graph = dict() # neighbpr ralation 
    self.edges = list() # set of edge that program can save
    self.ops = {
    '+' : operator.add,
    '-' : operator.sub}

  def addSample(self, edge):
    u, v = edge
    self.t = self.t + 1

    if self.sampleEdge((u,v)):

      self.edges.append((u,v)) # Add edge to edge samples
      self.addNeighbor((u, v)) # Add neighbor for both u and v
      self.updateCounter('+', (u,v)) # Add counters

  def sampleEdge(self, edge): # Decide wether to add edge to edge set
    u, v = edge

    if self.t <= self.M: # condition that edge set is not full
      return True
    elif self.flipBiasedCoin(self.M/self.t): # condition that set is full so flip a coin to decide
      
      random_index = random.randint(0, self.M - 1) # select one edge to delete
      (ud,vd) = self.edges.pop(random_index)

      self.deleteNeighbor((ud, vd)) # delete neighbor for both u and v
      
      self.updateCounter('-', (ud,vd)) # Update global and local triangles
      return True
    return False

  def flipBiasedCoin(self, p): # flip a coin
    if random.random() < p:
      coin = True
    else:
      coin = False
    return coin

  # Updating the global and local counter which is 
  # used to approximate the number of global and local triangels
  def updateCounter(self, op, edge):
    u,v = edge

    if not u in self.Graph or not v in self.Graph:
      return

    u_neighbors = self.Graph[u]
    v_neighbors = self.Graph[v]

    same_neighbors = u_neighbors & v_neighbors

    same_neighbors_count = len(same_neighbors)
    if same_neighbors_count == 0:
      return

    #Updating counter
    if v not in self.local_count:
      self.local_count[v] = 0
    if u not in self.local_count:
      self.local_count[u] = 0

    # Add neighbors count to global and local[vertice in edge]
    self.global_count = self.ops[op](self.global_count, same_neighbors_count)
    self.local_count[u] = self.ops[op](self.local_count[u], same_neighbors_count)
    self.local_count[v] = self.ops[op](self.local_count[v], same_neighbors_count)

    # Add 1 to the neighboring vertices of u and v
    for c in same_neighbors:
      if c not in self.local_count:
        self.local_count[c] = 0
      self.local_count[c] = self.ops[op](self.local_count[c], 1)
  
  # Estimate Triangles
  def estimateTriangles(self):
    return max(1,int(self.t*(self.t-1)*(self.t-2)/(self.M*(self.M-1)*(self.M-2)))) * self.global_count

  def addNeighbor(self ,edge):
    u, v = edge

    if u in self.Graph:
        self.Graph[u].add(v)
    else:
        self.Graph[u] = set([v])
    if v in self.Graph:
        self.Graph[v].add(u)
    else:
        self.Graph[v] = set([u])

  def deleteNeighbor(self, edge):
    u, v = edge

    if u in self.Graph:
      self.Graph[u].remove(v)
      if len(self.Graph[u]) == 0:
        del self.Graph[u]
    if v in self.Graph:
      self.Graph[v].remove(u)
      if len(self.Graph[v]) == 0:
        del self.Graph[v]



### Run the program

In [14]:
edges = set()
with open('web-NotreDame.txt') as f:
    for index, line in enumerate(f):
      if index < 5:
        continue
      edge = tuple(sorted(map(int, line.strip().split())))
      if edge[0] == edge[1]:
        continue
      edges.add(edge)
f.close()

In [22]:
#print("Number of edges:", index)
print("Number of Triangels:", 8910005)
for M in [10000, 20000, 40000, 50000, 70000, 80000, 100000]:
    print("edge set size", M)
    triestBase = TriestBase(M)
    start_time = time.time()
    for index, edge in enumerate(edges):
        triestBase.addSample(edge)
    print(" Time:", time.time()-start_time, "s")
    print(" Triangles:", triestBase.estimateTriangles())

Number of Triangels: 8910005
edge set size 10000
 Time: 1.1699986457824707 s
 Triangles: 10366392
edge set size 20000
 Time: 1.459001064300537 s
 Triangles: 7125800
edge set size 40000
 Time: 2.0369999408721924 s
 Triangles: 8805270
edge set size 50000
 Time: 2.660999298095703 s
 Triangles: 9150529
edge set size 70000
 Time: 3.3549985885620117 s
 Triangles: 8318528
edge set size 80000
 Time: 3.369999647140503 s
 Triangles: 9171250
edge set size 100000
 Time: 4.167998790740967 s
 Triangles: 8708875


### Questions

1. What were the challenges you have faced when implementing the algorithm?

Because I really don't know the paper's background, it is difficult for me to understand the paper at first. 

2. Can the algorithm be easily parallelized? If yes, how? If not, why? Explain.

No. The algorithm use the variable and sets from last step. Therefore, we can't parallelized this kind of linear funciton.

3. Does the algorithm work for unbounded graph streams? Explain.

Yes, it does. The algorithm is not bound to the number of elements in the stream, cause the algorithm only maintains a set of elements.  

4. Does the algorithm support edge deletions? If not, what modification would it need? Explain.

No, it does not. But I find that the last algorithm in the paper, Trièst-FD will uses the refined version of Reservoir Sampling called Random Pairing to accomplish this. 